In [30]:
from mexc_sdk import Spot
import os 
from dotenv import load_dotenv
import pandas as pd
import numpy as np
from datetime import datetime

In [31]:
now = datetime.now().date()

In [32]:
symbol = input("Symbol = ")
folder_save = f"../../data/{symbol}"

is_quick_get = input("is quick get? (y/n)").strip().lower() == "y"
if is_quick_get:
    start_date = f"{now.year}-01-01"
else:
    start_date = input("start date? (YYYY-MM-DD)").strip().lower()

to_date = str(now)

is_get_now = input("Get now? (y/n)").strip().lower()[0] == "y"
concanate_with_old = input("concanate with old data? (y/n)").strip().lower()[0] == "y"

interval = input("""interval = 
                 
                "1m" - "1 minute"
                "5m" - "5 minute"
                "15m" - "15 minute"
                "30m" - "30 minute"
                "60m" - "60 minute"
                "4h" - "4 hour"
                "1d" - "1 day"
                "1W" - "1 week"
                "1M" - "1 month"
                 """).strip()

In [33]:
load_dotenv(dotenv_path="./apikey.txt")

True

In [34]:
access_key = os.getenv("access_key")
secret_key = os.getenv("secret_key")

In [35]:
spot = Spot(api_key=access_key, api_secret=secret_key)

In [36]:
data_from_mexc = spot.klines(symbol.upper() + "USDT", interval=interval, 
    # options={
    #     "startTime": start_date,
    #     "endTime": to_date
    # }
    )

In [37]:
df_origin_mexc = pd.DataFrame(data_from_mexc)

In [38]:
df_origin_mexc.columns

RangeIndex(start=0, stop=8, step=1)

In [39]:
df_origin_mexc["Date"] = pd.to_datetime(df_origin_mexc.iloc[:, 0], unit="ms")

In [40]:
mapping_prices = {1: "Open",
                  2: "High",
                  3: "Low",
                  4: "Close",
                  5: "Volume", }

In [41]:
for index in mapping_prices:
    df_origin_mexc[mapping_prices[index]] = df_origin_mexc.iloc[:, index]

In [42]:
df_from_mexc = df_origin_mexc[['Open', 'High',
                'Low', 'Close',     
                'Volume', 'Date']]

In [43]:
now = datetime.now().date()

In [44]:
now

datetime.date(2024, 12, 2)

In [45]:
df_from_mexc

,Open,High,Low,Close,Volume,Date
0,29901.71,29998.99,29622.24,29794,1956.832041,2023-07-22
1,29794,30350,29730.01,30083.76,2999.126007,2023-07-23
2,30083.76,30099.58,28867,29176.5,5278.560984,2023-07-24
3,29176.5,29373.67,29047.07,29228.91,3256.319756,2023-07-25
4,29228.91,29688,29098,29351.95,4251.406055,2023-07-26
...,...,...,...,...,...,...
495,95863.12,96564,94640,95643.97,9872.55541859,2024-11-28
496,95643.97,98619.99,95364.99,97460.01,8612.00588688,2024-11-29
497,97460.01,97463.95,96092.01,96407.99,4571.59970941,2024-11-30
498,96407.99,97836,95693.88,97185.17,9115.36369784,2024-12-01


In [46]:
if not is_get_now:
    df_from_mexc = df_from_mexc[df_from_mexc["Date"] != str(now)]

In [47]:
if concanate_with_old:
    old_df = pd.read_csv(f"{folder_save}/{symbol.lower()}_to_now_{interval.lower()}.csv")
    old_df["Date"] = pd.to_datetime(old_df["Date"])
    old_df = old_df[['Open', 'High',
                'Low', 'Close',     
                'Volume', 'Date']]
    max_date_in_old_df = pd.to_datetime(old_df["Date"].max().date())
    df_today = df_from_mexc[df_from_mexc["Date"] > max_date_in_old_df]
else:
    df_today = df_from_mexc


In [48]:
df_today["Date"] = df_today["Date"].dt.date
if concanate_with_old:
    old_df["Date"] = old_df["Date"].dt.date
    df = pd.concat([old_df, df_today], ignore_index=True)
else:
    df = df_today
df["Date"] = pd.to_datetime(df["Date"])

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_65944\1533491661.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_today["Date"] = df_today["Date"].dt.date


In [49]:
df

,Open,High,Low,Close,Volume,Date
0,465.864014,468.174011,452.421997,457.334015,21056800.0,2014-09-17
1,456.859985,456.859985,413.104004,424.440002,34483200.0,2014-09-18
2,424.102997,427.834991,384.532013,394.79599,37919700.0,2014-09-19
3,394.673004,423.29599,389.882996,408.903992,36863600.0,2014-09-20
4,408.084991,412.425995,393.181,398.821014,26580100.0,2014-09-21
...,...,...,...,...,...,...
3724,91965.16,97208.21,91792.14,95863.12,9496.180183,2024-11-27
3725,95863.12,96564.0,94640.0,95643.97,9872.555419,2024-11-28
3726,95643.97,98619.99,95364.99,97460.01,8612.005887,2024-11-29
3727,97460.01,97463.95,96092.01,96407.99,4571.599709,2024-11-30


In [50]:
symbol = symbol.replace(".", "_").lower()

In [51]:
min_date = df["Date"].min().date()
max_date = df["Date"].max().date()

os.makedirs(folder_save, exist_ok=True)
file_name = f"{folder_save}/{symbol}_to_now_{interval.lower()}.csv"

In [52]:
file_name = file_name.replace("-", "_")

In [53]:
file_name

'../../data/btc/btc_to_now_1d.csv'

In [54]:
df.to_csv(file_name, index=False)